In [ ]:
import gradio as gr 
import glob
import csv
import os
import json
import pandas as pd
import tempfile
from keras.models import model_from_json

In [ ]:
def get_features(file, dictionary=dict()):
    interested_events = ['compiled','compiled successfully',"ran tests","modified"]
    quantified_events=["test_results","total_time"]
    sub_events=["add_lines","change","remove_lines",""]
    csv_fields=['Student',"total_time",'compiled','compiled successfully',"ran_tests","test_results","modified","add_lines","change","remove_lines","average_test_results","paste_lines","max_pastes",'average_paste','cps','breaks']
    print(file.name)
    data=json.load(file)
    max_chars=0
    sum_chars=0
    if len(data)!=0 and type(data) is not list:
        for majorkey, subdict in data.items():
            #print('majorkey time=>',subdict.keys())
            for subkey, value in subdict.items():
                #print('Subkey=>', subkey, 'Value=>',value)
                if subkey =='events':
                    for i in range(len(value)):
    
                        if value[i]['text'] in interested_events:
                            
                            if value[i]['text']=='compiled':
                                dictionary['compiled']=1+dictionary.get('compiled',0)
                            elif value[i]['text']=='compiled successfully':
                                dictionary['compiled']=1+dictionary.get('compiled',0)
                                dictionary['compiled successfully']=1+dictionary.get('compiled successfully',0)
                            elif value[i]['text']=='ran tests':
                                dictionary['ran_tests']=1+dictionary.get('ran_tests',0)
                                if value[i]['test_results'][:-3]!='':
                                    dictionary['test_results']=int(value[i]['test_results'][:-3])+dictionary.get('test_results',0)
                                    #print((value[i]['test_results'][:-3]))
                            elif value[i]['text']=='modified':
                                dictionary['modified']=1+dictionary.get('modified',0)
                                if "diff" in value[i]:
                                    if  (value[i]['diff'] is None) and (type(value[i]['diff']) is 'list'):
                                        dictionary['breaks'] = dictionary.get('breaks',0)
                                    if  (value[i]['diff']!= None) and (type(value[i]['diff']) is not 'list'):
        
                                        if 'add_lines' in value[i]["diff"]:
                                            dictionary['add_lines']=len(value[i]['diff']['add_lines'])+dictionary.get('add_lines',0)
                                            if len(value[i]['diff']['add_lines'])>int(5):
                                                dictionary['paste_lines']=len(value[i]['diff']['add_lines'])+dictionary.get('paste_lines',0)
                                                for lines in value[i]['diff']['add_lines']:
                                                    if type(lines) is not bool:
                                                        sum_chars+=len(lines)
                                                        num_chars = len(lines)
                                                        if(num_chars>max_chars):
                                                            max_chars=num_chars
                                                dictionary['max_pastes']=max_chars
                                                
                                        elif 'remove_lines' in value[i]["diff"]:
                                            dictionary['remove_lines']=len(value[i]['diff']['remove_lines'])+dictionary.get('remove_lines',0)
                                        elif 'change' in value[i]["diff"]:
                                            dictionary['change']=len(value[i]['diff']['change'])+dictionary.get('change',0)
                                            if len(value[i]['diff']['change'])>int(5):
                                                dictionary['paste_lines']=len(value[i]['diff']['change'])+dictionary.get('paste_lines',0)
                                                for lines in value[i]['diff']['change']:
                                                    if type(lines) is not bool:
                                                        sum_chars+=len(lines)
                                                        num_chars = len(lines)
                                                        if(num_chars>max_chars):
                                                            max_chars=num_chars
                                                dictionary['max_pastes']=max_chars
                
                                        
                                  
                        #Total time spent on a problem
                if 'total_time' in subdict.keys():
                    #print('hello')
                    dictionary['total_time']=subdict['total_time']+dictionary.get('total_time',0)
                if 'modified' in dictionary and 'total_time' in dictionary and dictionary['total_time']>0:
                    dictionary['cps']=sum_chars/dictionary['total_time']
        if 'test_results' in dictionary:
            dictionary['average_test_results'] = dictionary['test_results']/(dictionary['ran_tests'])
        else:
            dictionary['average_test_results']=None
        if 'paste_lines' in dictionary:
            dictionary['avg_paste']=sum_chars/dictionary['paste_lines']
        else:
            dictionary['avg_paste']=None
        
        return dictionary

In [ ]:
import tempfile
def file_handler(file):
    file.seek(0)
    d=get_features(file)
    print(d)
    prediction = run_model(d)
    return prediction

In [ ]:

gui = gr.Interface(fn=file_handler, #callable function
                   inputs='file', #input format
                   outputs="text") #output format
#display the interface
gui.launch()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://54960.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


/tmp/student10160yl4b6tx.json
{'total_time': 37440, 'modified': 1, 'remove_lines': 8, 'cps': 0.0, 'average_test_results': None, 'avg_paste': None}


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- add_lines
- change
- compiled
- compiled successfully
- max_pastes
- ...

  warnings.warn(message, FutureWarning)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/gradio/routes.py", line 256, in run_predict
    fn_index, raw_input, username, session_state
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 546, in process_api
    predictions, duration = await self.call_function(fn_index, processed_input)
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 462, in call_function
    block_fn.fn, *processed_input, limiter=self.limiter
  File "/usr/local/lib/python3.7/dist-packages/anyio/to_thread.py", line 32, in run_sync
    func, *args, cancellable=cancellable, limiter=limiter
  Fil

In [ ]:
import numpy as np
from sklearn.naive_bayes import BernoulliNB
#from sklearn.preprocessing import MinMaxScaler
def run_model(dictionary):
    '''
    val =0.
    df = pd.DataFrame(dictionary,index=[0])
    df.fillna(0,inplace=True)
    print(df)
    #df = np.asarray(df).astype(np.float32)
    json_file = open('model_acctest_69.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model_acctest_69.h5")
    print("Loaded model from disk")
    predictions=loaded_model.predict(df)
    rounded = np.round(predictions,10) 
    print(predictions)
    x=predictions[0][0]
    #print('hello',float(predictions[0][0]))
    prediction = ('Not Plagiarised'if  x< val else 'Plagiarised')
    print(prediction)
    return prediction
    '''
   


    bnb1 = BernoulliNB(binarize=0)
    bnb1.fit(X_train, Y_train)
    y_pred = bnb1.predict(pd.DataFrame(dictionary,index=[0]))
    return 'Not Plagiarized' if y_pred==0 else 'Plagiarized'

In [ ]:
data = pd.read_csv('new_dataset_with_label.csv',index_col=[0])
data.fillna(0,inplace=True)
'''
json_file = open('model_acctest_69.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_acctest_69.h5")
print("Loaded model from disk")
#predictions=loaded_model.predict(data.iloc[:2339,:-1])
#dat=data.iloc[:2338,:-1]
#dat=data.iloc[:,:-1]
label=data.iloc[:2338,-1]
data = data.iloc[:2338,:-1]

#dat= np.asarray(data).astype(np.float32)

predictions=loaded_model.predict(data)
'''
data.fillna(0, inplace=True)
data["Label"] =data["Label"].astype("int64")
x = data.iloc[:2339,:-1]
y = data.iloc[:2339,-1]

In [ ]:
import imblearn
from collections import Counter
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.2)
smote_enn = imblearn.combine.SMOTEENN(random_state=0,sampling_strategy='minority')
x_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train,Y_train)
print(sorted(Counter(y_train_resampled).items()))

[(0, 569), (1, 635)]


In [ ]:
data

,total_time,compiled,compiled successfully,ran_tests,test_results,modified,add_lines,change,remove_lines,cps,paste_lines,max_pastes,average_test_results,avg_paste
Student,,,,,,,,,,,,,,
A2016\student1013,6894752.0,1441.0,1116.0,689.0,1349.0,38441.0,12365.0,32597.0,2023.0,0.003436,5202.0,120.0,1.957910,4.553441
A2016\student1016,37440.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,0.000000,0.0,0.0,0.000000,0.000000
A2016\student1021,9363152.0,2108.0,1278.0,787.0,1605.0,58550.0,19160.0,49003.0,6105.0,0.003997,7270.0,162.0,2.039390,5.147455
A2016\student1029,6949656.0,1775.0,1201.0,852.0,754.0,36461.0,15605.0,30332.0,2683.0,0.005040,7235.0,165.0,0.884977,4.840774
A2016\student1066,2784368.0,338.0,223.0,449.0,810.0,13924.0,6140.0,12169.0,369.0,0.004970,3988.0,139.0,1.804009,3.470160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B2016\student9945,588448.0,132.0,104.0,102.0,191.0,2219.0,1158.0,1775.0,52.0,0.003451,790.0,110.0,1.872549,2.570886
B2016\student9946,2989136.0,311.0,127.0,220.0,335.0,15501.0,12362.0,13196.0,1776.0,0.017313,9610.0,315.0,1.522727,5.385016
B2016\student9949,7787496.0,1961.0,1199.0,588.0,468.0,39601.0,9607.0,30431.0,3452.0,0.002364,3735.0,347.0,0.795918,4.929317
